In [ ]:
import polars as pl
from math import sqrt, atan2, pi, acos
import random
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path = kagglehub.dataset_download("konivat/tree-of-life")
dfs = []
for _filename_ in os.listdir(path):
    df = pl.read_csv(os.path.join(path, _filename_))
    dfs.append(df)

In [ ]:
_circles_ = []
for i in range(4): _circles_.append((0,0,1.0+3.0*random.random()))
_packed_       = rt.packCircles(sorted(_circles_, key=lambda x: x[2], reverse=True))
x0, y0, x1, y1 = _packed_[0][0] - _packed_[0][2], _packed_[0][1] - _packed_[0][2], _packed_[0][0] + _packed_[0][2], _packed_[0][1] + _packed_[0][2]
for _circle_ in _packed_:
    x0 = min(x0, _circle_[0] - _circle_[2])
    y0 = min(y0, _circle_[1] - _circle_[2])
    x1 = max(x1, _circle_[0] + _circle_[2])
    y1 = max(y1, _circle_[1] + _circle_[2])
#perc = max(x1-x0, y1-y0)*0.5
#x0, y0, x1, y1 = x0-perc, y0-perc, x1+perc, y1+perc

fit_within = (10.0, 5.0, 3.0) # fit the packed within this circle
x0 = min(x0, fit_within[0] - fit_within[2])
y0 = min(y0, fit_within[1] - fit_within[2])
x1 = max(x1, fit_within[0] + fit_within[2])
y1 = max(y1, fit_within[1] + fit_within[2])


def translateAndScaleCircles(circles, into_circle):
    x0, y0, x1, y1 = circles[0][0] - circles[0][2], circles[0][1] - circles[0][2], circles[0][0] + circles[0][2], circles[0][1] + circles[0][2]
    for _circle_ in circles:
        x0, y0 = min(x0, _circle_[0] - _circle_[2]), min(y0, _circle_[1] - _circle_[2])
        x1, y1 = max(x1, _circle_[0] + _circle_[2]), max(y1, _circle_[1] + _circle_[2])
    
    x_mid,   y_mid   = (x1 + x0)/2.0,            (y1 + y0)/2.0
    x_scale, y_scale = (x1 - x0)/into_circle[2], (y1 - y0)/into_circle[2]
    scale            = max(x_scale, y_scale)

    fit = []
    for _circle_ in circles:
        x = (_circle_[0] - x_mid)/scale + into_circle[0]
        y = (_circle_[1] - y_mid)/scale + into_circle[1]
        fit.append((x,y,_circle_[2]/scale))

    return fit

svg = [f'<svg x="0" y="0" width="500" height="500" viewBox="{x0} {y0} {x1-x0} {y1-y0} ">']
svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff"/>')
svg.append(f'<circle cx="{fit_within[0]}" cy="{fit_within[1]}" r="{fit_within[2]}" fill="none" stroke="#ff0000" stroke-width="0.1"/>')
for i in range(len(_packed_)):
    _circle_ = _packed_[i]
    svg.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#e0e0e0" stroke-width="0.1"/>')
_aligned_circles_ = translateAndScaleCircles(_packed_, fit_within)
for i in range(len(_aligned_circles_)):
    _circle_ = _aligned_circles_[i]
    svg.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#000000" stroke-width="0.1"/>')
svg.append('</svg>')
rt.tile([''.join(svg)])